In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")
test = pd.read_csv("0000000000002747_test_twitter_x_test.csv")

In [0]:
train

In [0]:
test

In [0]:
train.drop('tweet_id', inplace = True, axis = 1)
train.drop('airline_sentiment_gold', inplace = True, axis = 1)
train.drop('negativereason_gold', inplace = True, axis = 1)
train.drop('tweet_coord', inplace = True, axis = 1)
train.drop('tweet_created',inplace = True, axis = 1)

In [0]:
train

In [10]:
mood_count = train['airline_sentiment'].value_counts()
mood_count

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [11]:
train['airline'].value_counts()

United            2928
US Airways        2152
American          2078
Southwest         1817
Delta             1639
Virgin America     366
Name: airline, dtype: int64

In [12]:
sentiments=train.airline_sentiment
text=train.text
x_test=test.text
documents=[]
for i in range(len(sentiments)):
    documents.append((text[i],sentiments[i]))
documents[0]

('@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled',
 'negative')

In [20]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
from string import punctuation

In [23]:
import string
stops=set(stopwords.words('english'))
punctuations=list(string.punctuation)
stops.update(punctuations)
stops

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [0]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [0]:
def simplify_pos_tag(tag):
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('J'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    if tag.startswith('V'):
        return 'v'
    return 'n'

In [0]:
import re

In [0]:
def hasNumbers(text):
    return bool(re.search('[0-9]+',text))

In [0]:
def clean_text(text):
    result=[]
    for word in text.split():
        word=word.strip(punctuation)
        if ((word.lower() not in stops) and (not hasNumbers(word.lower())) and (len(word.lower())>3)):
            posTag=pos_tag([word.lower()])[0][1]
            lemmatized_word=lemmatizer.lemmatize(word.lower(),simplify_pos_tag(posTag))
            result.append(lemmatized_word)
    return result

In [31]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
documents_modified=[(clean_text(text),sentiment) for text,sentiment in documents]
x_test=[clean_text(text) for text in x_test]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
Y_train = [sentiment for text,sentiment in documents_modified]
X_train = [" ".join(text) for text,sentiment in documents_modified]
X_test = [" ".join(text) for text in x_test]

In [33]:
X_test

["americanair pull road on-hold since can't reach arpt",
 'americanair plane didn’t land identical bad condition accord metars',
 "southwestair can't believe many pay customer left high reason flight cancel flightlations monday",
 'usairways legitimately would rather driven cross country flown airway',
 'americanair still response great guy',
 'united developer fly tmrw morn layover earlier flight layover move',
 'usairways hello anyone',
 'usairways husainhaqqani husain shld protest well party member rehman malik delayed flight hour',
 'usairways likely flightaware say plane still durango depart',
 'americanair even give option hold line busy late flightr',
 'united announcement boarding address mobility disability require travel stuff preboard',
 'usairways really embarrass ask complimentary drink/snack detailed argue',
 'southwestair passport time trip could still photo thingsishouldknow ifeeldumb',
 'americanair delayed friend lisa pafe day costa rica issue update system',
 'southw

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
cv=CountVectorizer(max_features=2000,ngram_range=(1,2))
X_train_features = cv.fit_transform(X_train)
X_test_features = cv.transform(X_test)
clf=MultinomialNB()
clf.fit(X_train_features,Y_train)
pred=clf.predict(X_test_features)

In [36]:
clf.score(X_train_features,Y_train)

0.7961748633879782

In [0]:
np.savetxt("twitter_predictions.csv", pred , fmt = '%s')